<span style="color:darkblue;">
    <h2>
        Dapsys Importer Notebook
    </h2>
</span>

<img src="../imgs/background.jpg" width="800">

First time dealing with Jupyter notebooks? <a href="https://ukaachen-my.sharepoint.com/:w:/r/personal/ekutafina_ukaachen_de/Documents/Microneurography%20Data%20Analysis%20Team/Scripts%20documentation/Python_Jupyter_%20Manual%20for%20non-CS%20people.docx?d=wd12099523c094eafb83287b67ffc3308&csf=1&web=1&e=RPbYmq">Click here</a>

<span style="color:darkblue;">
    <h2>
        Data summary
    </h2> 
</span>
<i>(More information available in Ukaachen drive)</i> 

With Dapsys Software, we’re able to extract files in .csv format, suitable for further data analysis. In the following, we’ll cover the dataset that is being extracted and its corresponding usage. There are 4 main files extracted from dapsys:

<table>
    <tr>
        <td><center><h2 style="background-color:black; color:white;">Continuous recording</h2></center></td>
        <td><center><h2 style="background-color:black; color:white;">Main pulses</h2></center></td>
        <td><center><h2 style="background-color:black; color:red;">Template</h2></center></td>
        <td><center><h2 style="background-color:black; color:yellow;">Track times</h2></center></td>
    </tr>
    <tr>
        <td><img src="../imgs/recordings.png"height="200"></td>
        <td><img src="../imgs/pulses.png" height="200"></td>
        <td><img src="../imgs/template.png" height="200"></td>
        <td><img src="../imgs/track_times.png" height="200"></td>
    </tr>
    <tr>
        <td><center>General recording. Every odd row represents timestamp and every even row represents corresponding amplitude. </center></td>
        <td><center>Timestamps that correspond to electrical stimuli. </center></td>
        <td><center>30-point list/vector, that contains amplitude values. We would like to find the spikes that are similar to this template. </center></td>
        <td><center>These are timestamps that correspond to the part of the signals which we know it’s generated from electrical stimulus. </center></td>
    </tr>
</table>

<span style="color:darkblue;">
    <h2>Hiding or showing the code</h2>
</span>

Before starting with the data analysis, you can choose whether you want to see the actual code or not. If you want to hide the code, you can run the cell below. Bear in mind that hiding the code, could result in lack of focus and more difficult usage for beginners in Jupyter. Though, you can always hide/show the code. <b>If you appear to have just hidden code + restarted kernel (without any visibility of cells), no worries, you can just select this text-cell and move to the below one and run it.</b>

In [ ]:
from IPython.core.display import HTML, display, Image, clear_output
HTML('''
<style>.container { width:100% !important; } div.output_area {overflow-y: scroll;} div.output_area img {max-width: unset;} </style>
<script>code_show=true; 
function code_toggle() {
 if (code_show){$('div.input').hide();} else {$('div.input').show();}
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<p style="color:red"><strong>To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.</p></strong>''')

<span style="color:darkblue;">
<h2>Step 1: Define path of your 4 files</h2>
</span>

So you would want to have all of the four files within the same directory. Please note that for successful reading of the files, you must name corresponding files with endings as follows: pulses, track, template, recording. For instance: 

<img src="../imgs/naming.png">

After you run the cell, a text-box should appear, with a default value of a path. If you would like to assess new files, please make a separate folder in "Dapsys/data" folder, and then type in the path in the text box. 

In [ ]:
import pandas as pd
import plotly.express as px
from importers import DapsysImporter
from plotting import *

text_path = widgets.Text(description='Insert path:', value='../data/alternative_3')
out_path = widgets.Output()
widgets.VBox([text_path, out_path])

When you move to the next cell and run it, it will automatically try to gather data in necessary format for analysis. You should get an output which says that all four files are being read.

In [ ]:
dapsys = DapsysImporter(dir_path = text_path.value)
trck_dict = dapsys.extract_segment_idxs_times()
freq = round((dapsys.point_data[1][0] - dapsys.point_data[0][0]) * 1000, 5)
correct_signals = dapsys.get_action_potentials()

correct_signals_dict = convert_signals_to_dict(correct_signals, dapsys.main_pulses, freq, full_scale=False, template=dapsys.template, trck_dict=None)
template_dict = convert_template_to_dict(dapsys.template, freq)
traces = find_traces_amplitudes(correct_signals_dict+[template_dict])

<span style="color:darkblue;">
    <h2>
        Step 2: Define threshold for AP identification
    </h2>
</span>
Next step is to manually define a threshold for identifying an action potential. More on the actual method can be found in the documentation. Run the cell below to activate the possibility of choosing a threshold. By default it is 5. To choose a "good threshold value", after running the cell, you will be able to see the all the correct signals (generated from track times) and their corresponding shape, latency, segment in which they appeared (stimulus-to-stimulus window) and finally correlation with the <span style="color:red;">template.</span>

In [ ]:
graph1=go.FigureWidget(data=traces, layout=go.Layout(width=800, height=400, title=dict(text='Template and correct spikes'), 
                                                xaxis=dict(title='Time (ms)', range=[0, 3-freq]), 
                                                yaxis=dict(title='Amplitude', range=[-10, 10]), barmode='overlay'))

slider_templ = widgets.FloatSlider(description='Threshold:', min=1, max=10, step=0.05, value =5)
button_templ = widgets.Button(description='Find spikes')
widgets.VBox([graph1, slider_templ])

After you run the cell below this one, you'll extract the data in necessary format for visualization and analysis.

In [ ]:
threshold = slider_templ.value
print(f'Getting spikes with threshold of {threshold}...')
taps = dapsys.get_threshold_action_potentials(threshold)

taps_dict = convert_signals_to_dict(taps, dapsys.main_pulses, freq, full_scale=True, template=dapsys.template, trck_dict=trck_dict)
threshold_dict = convert_threshold_to_dict(threshold, 40000, freq)
sigs_dict = taps_dict+[threshold_dict]

amplitudes_list = dapsys.get_raw_signal_split_by_stimuli()
times_list = dapsys.get_raw_timestamp_split_by_stimuli()
tap_corrs = [tap['corr'] for tap in taps_dict]
real_corrs = [tap['corr'] for tap in correct_signals_dict]

<span style="color:darkblue;">
    <h2>
        Visualization 1: Main pulse divided signal recording, with APs
    </h2>
</span>

This is the main visualization of the actual recording, which is being split by segment. Here, "Segment" corresponds to time period between the two electrical stimuli. When you run the following cell, you will firstly see the first segment. There is a possibility to use slider to switch to some other segment. Manually found APs are colored in green, whereas the ones from electrical stimulus are colored in yellow with 👌 symbol. Using the list on the right, one can examine the correlation and time latency to an electrical stimulus. 

In [ ]:
segments_main = widgets.IntSlider(value=1, min=1, max=len(amplitudes_list), step=1, description='Segment:', continuous_update=False)
container_main = widgets.HBox(children=[segments_main])
g = big_plot(dapsys, sigs_dict, amplitudes_list, freq)
fp = functools.partial(update_big_plot, g=g, amplitudes_list = amplitudes_list, 
                       segments_main = segments_main, freq = freq, sigs_dict = sigs_dict)
segments_main.observe(fp, names='value')
display(widgets.VBox([container_main, g]))

<span style="color:darkblue;">
    <h2>
        Visualization 2: Histogram/Distribution plot of manually found and correct APs
    </h2>
</span>

Now, given the <span style="color:green;">manually found APs</span> and <span style="background-color:gray; color:yellow;">"correct ones"</span> (from track_times.csv file)</span>, we can use following visualizations for further examination:<br/>

- Using histogram and boxplot of correlation values between APs 
- Directly plotting the signal

In addition, a slider below the plots, can be used to extract higher correlation points. To do so, just slide the slider to a higher point: Both plots should change accordingly. 

In [ ]:
fig1, fig2 = histo_signal_plot(dapsys, tap_corrs, real_corrs, freq, taps, trck_dict)


out_mod = widgets.Output()
button_mod = widgets.Button(description='Modify threshold!', layout=widgets.Layout(width='200px', height='40px'))
slider_mod = widgets.FloatSlider(description='Correlation:', min=0.3, max=1, step=0.05)
fp2 = functools.partial(histo_signal_plot_update, out_mod=out_mod, slider_mod=slider_mod, dapsys=dapsys, 
                         freq=freq, trck_dict=trck_dict, 
                         fig1=fig1, fig2=fig2, tap_corrs=tap_corrs, taps=taps)

button_mod.on_click(fp2)
display(widgets.VBox(children=[fig1, fig2, out_mod, slider_mod, button_mod]))
display(HTML('<i>(Could be slow in case of too many signals)<i>'))

Next cell is being ran for the purpose of data wrangling for visualizing remaining useful plots. 

In [ ]:
track_df = pd.DataFrame(dict_to_df(correct_signals, dapsys.template.signal_template), columns=['segment_id', 'Time (s)', 'Correlation']).astype({'segment_id': 'int32'})
thresh_df = pd.DataFrame(dict_to_df(taps, dapsys.template.signal_template), columns=['segment_id', 'Time (s)', 'Correlation']).astype({'segment_id': 'int32'})
idxs = thresh_df.segment_id.unique()
min_idx = min(idxs); max_idx = max(idxs)
mapping = {l:m for m,l in enumerate(idxs)}

<span style="color:darkblue;">
    <h2>
        Visualization 3: Delay - Correlation plot (correct signals)
    </h2>
</span>

In the following cell, you will get a visualization that explains latency-correlation relationship of the <span style="background-color:gray;color:yellow;">correct action potentials</span>.
The lighter the point - later it appears (In terms of the segment index). 

In [ ]:
px.scatter(track_df, labels="segment_id", x="Time (s)", y="Correlation", color='segment_id', hover_data=['segment_id'], range_x=[0,4], range_y=[0,1], title="Correct signals correlation").show()

<span style="color:darkblue;">
    <h2>
        Visualization 4: Delay - Correlation plot (manually found APs)
    </h2>
</span>

Now, you will be able to assess the same relationship, but with <span style="color:green;">manually found APs</span>.

By default, only the first segment is being selected, and other ones faded. To change the segment, select the slider and click "Set active segment". To see all of the points more clearly, click "Set all segments active".

In [ ]:
gg = reaction_plot_threshold(thresh_df)
segment_slider_act = widgets.IntSlider(description='Segment:', min=min_idx, max=max_idx)
button_act1 = widgets.Button(description='Set active segment!', layout=widgets.Layout(width='200px', height='40px'))
button_act2 = widgets.Button(description='Set all segments active!', layout=widgets.Layout(width='200px', height='40px'))
out_act2 = widgets.Output()

fp3 = functools.partial(delay_plot_set_active_segment, 
                        out_act = out_act2, g = gg, segment_slider_act = segment_slider_act, 
                        mapping = mapping, idxs = idxs)

fp4 = functools.partial(delay_plot_set_all_segments_active,
                        out_act = out_act2, g = gg, idxs = idxs)

button_act1.on_click(fp3)
button_act2.on_click(fp4)
container = widgets.HBox([button_act1, button_act2])
display(widgets.VBox([gg, out_act2, segment_slider_act, container]))

<span style="color:darkblue;">
    <h2>
        Step 3 (optional): Save manually found APs in excel table
    </h2>
</span>

Finally, you can extract these <span style="color:green;">manually found APs</span>, in a table, ready for excel extraction. To extract it as an excel file, type in the textbox, name of the file that you would like to use. After clicking "Save table as an excel file", you can access it, from the same folder that this notebook is in. 

In [ ]:
text_act = widgets.Text(description='Insert path:', value='test')
out_act3 = widgets.Output()
fp5 = functools.partial(save_excel, out_act = out_act3, thresh_df = thresh_df, text_act = text_act)
button_act3 = widgets.Button(description='Save table as excel file', layout=widgets.Layout(width='200px', height='40px'))
button_act3.on_click(fp5)

display(HTML("<br/>"))
display(thresh_df[['segment_id', 'Time (s)', 'Correlation']])
display(widgets.VBox([text_act, out_act3, button_act3]))